# Baseline ICASSP 2024 Grand Challenge

This tutorial walks through the process of running the ICASSP Grand Challenge baseline using the shell Interface


## Setting the Location of the Project

For convenience, we are setting an environment variable with the location of the root working directory of the project. This variable will be used in various places throughout the tutorial. Please change this value to reflect where you have installed this notebook on your system.

In [1]:
import os
os.environ["NBOOKROOT"] = os.getcwd()
os.getenv("NBOOKROOT")

'/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines'

## Installing Clarity Tag v0.4.1
We first need to install the Clarity package. For this we use the tag version for the challenge.

In [19]:
from IPython.display import clear_output

import os
import sys

print("Cloning git repo...")
!git clone --depth 1 --branch v0.4.1 https://github.com/claritychallenge/clarity.git
%cd clarity

%pip install -e .

sys.path.append(f'{os.getenv("NBOOKROOT")}/clarity')

clear_output()
print("Repository installed")

Repository installed


## Get the demo data

We will be using music audio and listener metadata.

In [ ]:
!gdown 1mZdh_BeUrFzGQrG6vjJnU97EDfFooLLN
!mv demo_data_icassp_2024.tar.xz recipes/cad_icassp_2024/baseline
%cd recipes/cad_icassp_2024/baseline
!tar -xvf demo_data_icassp_2024.tar.xz

clear_output()
print("Data installed")

## Changing working Directory

Next, we change working directory to the location of the shell scripts we wish to run.

In [ ]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad_icassp_2024/baseline

## Inspecting Existing Configuration

All of the included shell scripts take configurable variables from the yaml files in the same directory as the shell script.Typically these are named <code>config.yaml</code>, however, other names may be used if more than one shell script is in a directory.

We can inspect the contents of the config file using <code>!cat</code>:

```yaml
path:
  root: ???
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.root}/audio/at_mic_music
  gains_file: ${path.metadata_dir}/gains.json
  head_positions_file: ${path.metadata_dir}/head_positions.json
  listeners_file: ${path.metadata_dir}/listeners.train.json
  music_file: ${path.metadata_dir}/at_mic_music.train.json
  scenes_file: ${path.metadata_dir}/scenes.train.json
  scene_listeners_file: ${path.metadata_dir}/scene_listeners.train.json
  exp_folder: ./exp # folder to store enhanced signals and final results

sample_rate: 44100
remix_sample_rate: 32000
HAAQI_sample_rate: 24000

nalr:
  nfir: 220
  sample_rate: ${sample_rate}

apply_compressor: False
compressor:
  threshold: 0.35
  attenuation: 0.1
  attack: 50
  release: 1000
  rms_buffer_size: 0.064

soft_clip: False

separator:
  model: demucs   # demucs or openunmix
  device: ~

evaluate:
  set_random_seed: True
  small_test: False
  batch_size: 1  # Number of batches
  batch: 0       # Batch number to evaluate

# hydra config
hydra:
  run:
    dir: ${path.exp_folder}
```

## Running Enhancement

We are now ready to run the `enhance.py` python script. However, the standard configuration is designed to work with the full dataset. We can redirect the script to the correct folders by overriding the appropriate configuration parameters.

In [40]:
!python enhance.py path.root={os.environ['NBOOKROOT']}/clarity/recipes/cad_icassp_2024/baseline/demo_data_icassp_2024/cad_icassp_2024 'path.listeners_file=${path.metadata_dir}/listeners.valid.json' 'path.scenes_file=${path.metadata_dir}/scenes.valid.json' 'path.scene_listeners_file=${path.metadata_dir}/scene_listeners.valid.json'

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
no viable alternative at input '{os.environ['
See https://hydra.cc/docs/1.2/advanced/override_grammar/basic for details

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


Now we have the enhanced output. Below, we can load and play the audio to listen to examples of the results.

In [ ]:
from os import listdir
from os.path import isfile, join
from pathlib import Path
from scipy.io import wavfile

import IPython.display as ipd

audio_path = Path("exp/enhanced_signals")
audio_files = [f for f in audio_path.glob('*/*/*') if f.suffix == '.wav']

for file_to_play in audio_files:
  sample_rate, signal = wavfile.read(file_to_play)
  # Take only 30 seconds
  if signal.shape[-1] == 1:
    signal = signal[30*sample_rate:60*sample_rate, :]
  else:
    signal = signal[30*sample_rate:60*sample_rate]
  print(file_to_play.name)
  ipd.display(ipd.Audio(signal.T, rate=sample_rate))

Now that we have enhanced audios we can use the evaluate recipe to generate HAAQI scores for the signals. The evaluation is run in the same manner as the enhancement script.

In [ ]:
%%shell
python evaluate.py \
path.root=../cadenza_data_demo/cad1

We hope that this tutorial has been useful and has explained the process for using the recipe scripts using the Hydra configuration system. This approach can be applied to all of the recipes that are included in the repository.